# 📚 Project Overview

The goal of this project is to fine-tune large language models (LLMs) to generate Statements of Purpose (SOPs) in a specific author's writing style, and later extend it to generate both SOPs and stylistic reasoning explanations.

In this notebook, we focus on two major phases:

### 1. Fine-Tuning Mistral-7B on Custom SOP Instructions
- **Prepare a custom instruction dataset** from the original SOPs, formatting them with `Instruction`, `Input`, and `Response` sections.
- **Fine-tune** a 4-bit quantized Mistral-7B-Instruct-v0.2 model using Low-Rank Adaptation (LoRA).
- **Train** the model to generate complete SOPs when given an instruction prompt.
- **Evaluate** the fine-tuned model by running inference on new prompts and verifying that the SOPs generated were stylistically complete and coherent.

### 2. Creating an Enhanced SOP + Reasoning Dataset
- **Generate stylistic reasoning explanations** for each SOP using a pre-trained LLM (Mistral-7B).
- **Enhance the dataset** by combining each SOP with its corresponding explanation of writing style (tone, structure, word choice).
- **Save the final enhanced dataset** for use in **training Model 2**, which will be designed to generate both the SOP and its reasoning together.

✅ By the end of this notebook:
- We have a **fine-tuned SOP generation model**.
- We have created a **new structured dataset (SOP + reasoning)** for future fine-tuning of Model 2.


### 📦 Installing Required Libraries

Install the necessary libraries for working with Hugging Face models and diffusion models:
- `transformers`: For loading and using pre-trained large language models (LLMs).
- `accelerate`: For efficient multi-device (GPU/TPU/CPU) training and inference.
- `diffusers`: For working with diffusion models (if needed later for generation tasks).

✅ Run the command below to install all required packages:

In [ ]:
!pip install transformers accelerate diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 857.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
login()

### ✨ Automatic Instruction Generation from SOPs (LLM-Powered)

In this section:
- We use a Mistral-7B-Instruct model to **automatically generate instructions** based on existing SOPs.
- For each SOP:
  - We create a prompt asking the LLM to generate a one-line instruction starting with:  
    **"Write me an SOP for..."**
  - The instruction should capture the target degree and any focus area (academic/career).
- We store the outputs in a structured YAML format with fields:
  - `"instruction"`: The generated instruction.
  - `"input"`: (empty string, for future use if needed).
  - `"output"`: The original SOP text.

#### Key details:
- **Partial checkpointing**: Save intermediate results every 25 SOPs to avoid losing progress if the notebook crashes.
- **Error handling**: If the LLM fails for a SOP, a fallback instruction is assigned.
- **Efficiency**: `tqdm` is used to show progress during batch processing.

✅ Final output is saved as `/content/drive/MyDrive/manasa_sops_llm_prompts.yml`

In [ ]:
!pip install transformers

from transformers import pipeline
import pandas as pd
import yaml
import time
from tqdm import tqdm

# Load your CSV
df = pd.read_csv("/content/drive/MyDrive/manasa_cleaned_documents_final.csv")

# LLM-powered instruction generator
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device_map="auto",pad_token_id=2)

def smart_llm_prompt(text):
    preview = " ".join(text.strip().split("\n")[:3])[:500]
    prompt = (
        "Read the beginning of this SOP and generate a one-line instruction that starts with: "
        "'Write me an SOP for...' — Include the degree the applicant is applying for and, if possible, the academic or career focus.\n\n"
        f"SOP: {preview}\nInstruction:"
    )
    response = generator(prompt, max_new_tokens=50, temperature=0.7, top_p=0.9, top_k=50, repetition_penalty=1.2)
    return response[0]["generated_text"].split("Instruction:")[-1].strip()


# Generate YAML entries
records = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Generating prompts"):
    sop_text = row["Extracted Text"]
    print(f"🧪 Starting SOP #{i + 1}...")

    try:
        start = time.time()
        instruction = smart_llm_prompt(sop_text)
        duration = round(time.time() - start, 2)
        print(f"✅ Prompt generated in {duration}s — {instruction}")
    except Exception as e:
        print(f"⚠️ Error at SOP #{i}: {e}")
        instruction = "Write me an SOP for a graduate program."

    records.append({
        "instruction": instruction,
        "input": "",
        "output": sop_text.strip()
    })

    # Save partial progress every 25 SOPs
    if i % 25 == 0 and i > 0:
        with open("/content/drive/MyDrive/partial_manasa_sops.yml", "w") as f:
            yaml.dump(records, f, allow_unicode=True, sort_keys=False)
        print(f"💾 Saved partial results at SOP #{i + 1}")

# Save the final YAML
with open("/content/drive/MyDrive/manasa_sops_llm_prompts.yml", "w") as f:
    yaml.dump(records, f, allow_unicode=True, sort_keys=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

✅ Prompt generated in 1.25s — Write me an SOP for pursuing Master's Degree in Computer Science at XYZ University, focusing on Artificial Intelligence research.
🧪 Starting SOP #558...


Generating prompts: 100%|█████████▉| 558/559 [11:18<00:01,  1.13s/it]

✅ Prompt generated in 0.85s — Write me an SOP for getting admitted into a Master's program in Computer Science.
🧪 Starting SOP #559...


Generating prompts: 100%|██████████| 559/559 [11:19<00:00,  1.22s/it]

✅ Prompt generated in 0.97s — Write me an SOP for pursuing a Master's Degree in Computer Science focusing on Software Engineering.


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manyli

### 🔄 Convert YAML Dataset to JSONL Format

In this section:
- We load the generated YAML file (`manasa_sops_llm_prompts.yml`) containing SOP instructions and outputs.
- We convert the dataset into `.jsonl` (JSON Lines) format, where:
  - Each line is a separate JSON object with `instruction`, `input`, and `output` fields.
- JSONL format is useful for training LLMs, fine-tuning tasks, or sharing datasets with Hugging Face Datasets and other libraries.

✅ The final converted file is saved as: `/content/drive/MyDrive/manasa_sops_final.jsonl`

In [ ]:
import yaml
import json

with open("/content/drive/MyDrive/manasa_sops_llm_prompts.yml") as f:
    data = yaml.safe_load(f)

with open("/content/drive/MyDrive/manasa_sops_final.jsonl", "w") as f:
    for row in data:
        json.dump(row, f)
        f.write("\n")

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 📚 Load Instruction Dataset for Fine-Tuning

In this section:
- We load the newly created `.jsonl` file (`manasa_sops_final.jsonl`) using Hugging Face `datasets`.
- Each record contains:
  - `"instruction"`: the generated instruction (e.g., "Write me an SOP for...").
  - `"input"`: (empty string, not used here but kept for compatibility).
  - `"output"`: the full SOP text.

✅ This prepares the dataset for supervised fine-tuning (SFT) where the model learns to generate an SOP given an instruction.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from transformers import DataCollatorForSeq2Seq

# Load your dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/manasa_sops_final.jsonl", split="train")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': "Write me an SOP for pursuing a Master's Degree in Sustainable Architecture.", 'input': '', 'output': 'STATEMENT OF PURPOSE The future belongs to those and only those who aspire to not snatch away the life out of the living, the golden terrains out from the womb of the dearest Mother Earth. Sustainability, climatechange, bio mimicking, green architecture and more, interchangeably, the vocabulary of modernity that strives to strike chords with bringing a difference in existing consumer habits has unknowingly always been a part of my personality and lifestyle. My name is Mythri, meaning friendship. In my country: India, friendships signifies the scared relationship of man with all living forms(flora fauna) and the many elements of Earth (Earth, water, air, fire space). I grew up with the same love, considerability and accountability for life voicing out my opinions strongly. As a student, I was incredibly impressed with the many forms of nature, I wore an inquisitive scen

### 🛠️ Format Dataset into Instruction-Input-Response Format

In this step:
- Each example from the dataset is reformatted into a single `"text"` field following an instruction-tuning style format:

Instruction:
{instruction}

Input:
{input}

Response:
{output}

- Even though the `"input"` field is empty in this case, keeping it structured like this ensures compatibility with standard supervised fine-tuning (SFT) setups.

✅ The dataset is updated so that each example has a clean, standardized `"text"` field, ready for tokenization and model training.


In [ ]:
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)


Map:   0%|          | 0/559 [00:00<?, ? examples/s]

### 🧹 Tokenize the Formatted Dataset

In this step:
- We load the tokenizer associated with the base model (`mistralai/Mistral-7B-Instruct-v0.2`).
- We set the pad token to the EOS (end-of-sequence) token for consistency.
- Each example's `"text"` field is **tokenized** into model input IDs with:
  - **Padding** to a fixed maximum length (`max_length=1024` tokens).
  - **Truncation** if the text is too long.
- We use `batched=True` during mapping for efficient tokenization across multiple examples at once.

✅ This prepares the dataset for input into the model during fine-tuning.

In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

### 🎯 Load Fine-Tuned Mistral-7B Model with LoRA Adapter (for Inference)

In this step:
- **Load the tokenizer** from the base Mistral-7B-Instruct model.
- **Load the base model** in efficient 4-bit quantized mode (`nf4` quantization) using `BitsAndBytesConfig`.
- **Apply the LoRA adapter** trained earlier (`/content/drive/MyDrive/mistral-manasa-lora`) to the base model.
- This combines the pre-trained base model and the fine-tuned lightweight adapter into a ready-to-use model for generation.

✅ The model is now fully loaded, memory-efficient, and ready for inference on new prompts.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch

base_model_path = "mistralai/Mistral-7B-Instruct-v0.2"
lora_model_path = "/content/drive/MyDrive/mistral-manasa-lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

# Load base model in 4-bit mode
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load LoRA-adapted model
model = PeftModel.from_pretrained(base_model, lora_model_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### ⚙️ Define Training Arguments

In this step:
- We configure the training settings using Hugging Face's `TrainingArguments` class.
- Key settings:
  - `output_dir`: Where the model checkpoints and logs will be saved.
  - `per_device_train_batch_size=1`: Small batch size (balanced by accumulation steps).
  - `gradient_accumulation_steps=4`: Simulate a larger effective batch size.
  - `num_train_epochs=3`: Train for 3 full passes through the dataset.
  - `learning_rate=2e-4`: A slightly higher learning rate, suitable for LoRA fine-tuning.
  - `fp16=True`: Use mixed precision (16-bit floating point) for faster and memory-efficient training.
  - `logging_steps=10`: Log training metrics every 10 steps.
  - `save_total_limit=2`: Keep only the 2 most recent model checkpoints.
  - `save_strategy="epoch"`: Save the model at the end of every epoch.
  - `report_to="none"`: Disable external logging (e.g., no WandB or TensorBoard by default).

✅ These arguments optimize training for limited memory environments while maintaining good checkpoint management and logging.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/mistral-manasa-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_total_limit=2,
    save_strategy="epoch",
    report_to="none"
)


### 🏋️ Initialize Trainer and Start Fine-Tuning

In this step:
- **Initialize the `Trainer`** from Hugging Face Transformers to manage the training loop.
- Components:
  - `model`: The base model + LoRA adapter combined.
  - `args`: Training arguments defined earlier (`training_args`).
  - `train_dataset`: The tokenized dataset of formatted instruction-response examples.
  - `data_collator`: `DataCollatorForLanguageModeling` with `mlm=False`, since we are fine-tuning for causal language modeling (not masked language modeling).

- **Start training** by calling `trainer.train()`.

✅ This will fine-tune the model to generate SOPs based on the provided instructions while following the author's style.

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,2.984800
20,2.782400
30,2.779800
40,2.677700
50,2.690500
60,2.639100
70,2.641600
80,2.640100
90,2.636600
100,2.658100


TrainOutput(global_step=417, training_loss=2.562814474677582, metrics={'train_runtime': 635.1145, 'train_samples_per_second': 2.64, 'train_steps_per_second': 0.657, 'total_flos': 7.280147544460493e+16, 'train_loss': 2.562814474677582, 'epoch': 2.9803220035778173})

### 💾 Save Fine-Tuned Model and Tokenizer

After training:
- Save the **LoRA-adapted model** to the specified directory.
- Save the **tokenizer** to the same directory to ensure consistent tokenization during inference.

✅ This ensures the fine-tuned model and tokenizer can be easily reloaded later for generating SOPs or further fine-tuning.

In [ ]:
model.save_pretrained("/content/drive/MyDrive/mistral-manasa-lora")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-manasa-lora")

('/content/drive/MyDrive/mistral-manasa-lora/tokenizer_config.json',
 '/content/drive/MyDrive/mistral-manasa-lora/special_tokens_map.json',
 '/content/drive/MyDrive/mistral-manasa-lora/tokenizer.model',
 '/content/drive/MyDrive/mistral-manasa-lora/added_tokens.json',
 '/content/drive/MyDrive/mistral-manasa-lora/tokenizer.json')

### 📦 Checkpoints Saved During Training

During fine-tuning, the model automatically saved intermediate checkpoints:
- `/content/mistral-manasa-lora/checkpoint-280`
- `/content/mistral-manasa-lora/checkpoint-417`

Each checkpoint contains:
- A snapshot of the model's weights at a specific training step.
- Optimizer and scheduler states (useful if resuming training).

✅ These checkpoints allow us to:
- Resume training from a specific point if needed.
- Select the best checkpoint for final model evaluation if required.

In [ ]:
/content/mistral-manasa-lora/checkpoint-280
/content/mistral-manasa-lora/checkpoint-417

### 🧠 Inference: Generate SOPs Using Fine-Tuned Model

In this final step:
- **Create a text-generation pipeline** using the fine-tuned model and tokenizer.
- **Provide a new prompt** formatted exactly like the training examples:
  - `### Instruction:` followed by the instruction.
  - `### Input:` (empty in this case).
  - `### Response:` where the model should start generating.
- **Set generation parameters**:
  - `max_new_tokens=500`: Limit response length.
  - `temperature=0.7`, `top_p=0.9`, `top_k=50`: Control diversity and creativity.
  - `repetition_penalty=1.2`: Discourage repetitive output.

✅ This generates a full SOP based on the given instruction, following the style the model learned during fine-tuning.

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "### Instruction:\nWrite me an SOP for an MS in AI.\n\n### Input:\n\n### Response:\n"

response = generator(
    prompt,
    max_new_tokens=500,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)[0]["generated_text"]

print(response)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

### Instruction:
Write me an SOP for an MS in AI.

### Input:

### Response:
Statement of Purpose I have always believed that the world is a combination of algorithms and patterns, all interconnected and evolving to meet the demands of humankind. The future of technology and its applications has been made clear with how Artificial Intelligence (AI) and Machine Learning (ML) have come together to transform industries by creating smart solutions to everyday problems. As a computer engineering graduate who believes in the power of innovation and ideas, I am motivated to study this domain further while specialising in Artificial Intelligence. With time, my interest towards this branch of technology grew stronger as more use cases started surfacing across various sectors. From automobile to transportation, from healthcare to retail, everything today has become smarter through the help of artificial intelligence. It was when I realised the true potential of studying this stream of informatio

### 🧠 Enhance SOP Dataset with LLM-Generated Reasoning

In this step:
- We load the original SOP dataset (`manasa_sops_final.jsonl`).
- For each SOP:
  - We create a prompt asking the LLM to generate 2–3 bullet points explaining the writing style choices (tone, structure, and word choices).
  - We append the explanation below the SOP text under the section "🧠 Explanation:".
- We save the enhanced dataset to a new `.jsonl` file (`manasa_sop_with_reasoning.jsonl`).

#### Key Components:
- **Model**: Free Hugging Face model `mistralai/Mistral-7B-Instruct-v0.2`.
- **Prompt format**: Clear instruction for the model to describe the writing style.
- **Post-processing**:
  - If explanation generation fails, a fallback placeholder is inserted.
  - Progress is tracked using `tqdm`.
  - Each entry is updated and saved line-by-line.

✅ This creates a richer dataset where each SOP now comes with an LLM-generated stylistic explanation, useful for evaluation, further fine-tuning, or analysis.

In [ ]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# 📌 Step 1: Load model and tokenizer (free Hugging Face model)
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 🚀 Step 2: Create pipeline (NO device arg since we used device_map)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 📂 Step 3: Read SOP dataset (.jsonl format)
input_path = "/content/drive/MyDrive/manasa_sops_final.jsonl"  # Replace with your path
output_path = "/content/drive/MyDrive/manasa_sop_with_reasoning.jsonl"

with open(input_path, "r") as infile:
    data = [json.loads(line) for line in infile]

# 🔍 Step 4: Format each prompt for reasoning
def format_reasoning_prompt(sop_text):
    return (
        "Read this SOP and explain its writing style choices in 2–3 bullet points. "
        "Focus on tone, sentence structure, and word choices.\n\n"
        f"SOP:\n{sop_text.strip()}\n\n🧠 Explanation:"
    )

# 🧠 Step 5: Generate and save SOPs with explanations
with open(output_path, "w") as outfile:
    for i, entry in enumerate(tqdm(data)):
        if "output" not in entry:
            print(f"⚠️ Skipping entry {i}: Missing 'output' field.")
            continue

        sop = entry["output"]
        reasoning_prompt = format_reasoning_prompt(sop)

        try:
            response = generator(
                reasoning_prompt,
                max_new_tokens=100,
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                repetition_penalty=1.2
            )[0]["generated_text"]

            explanation = response.split("🧠 Explanation:")[-1].strip()
            entry["output"] = f"{sop.strip()}\n\n🧠 Explanation:\n{explanation}"

        except Exception as e:
            print(f"⚠️ Error generating reasoning for entry {i}: {e}")
            entry["output"] = f"{sop.strip()}\n\n🧠 Explanation:\n[Could not generate reasoning]"

        outfile.write(json.dumps(entry) + "\n")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
  0%|          | 0/559 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 10/559 [00:35<33:40,  3.68s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 559/559 [35:36<00:00, 

### 🔄 Reformat Chat-Style Dataset into Structured SOP + Explanation Fields

In this step:
- We load the original chat-style dataset (`manasa_chatml_dataset.jsonl`) where:
  - The assistant's response combines the SOP and its explanation as a single block of text.
- For each example:
  - **Split** the assistant's response into:
    - `"sop"`: the main Statement of Purpose.
    - `"explanation"`: the stylistic reasoning (if available).
  - **Handle flexible splitting** using regular expressions to catch variations like:
    - `🧠 Explanation:`
    - `Explanation:`
    - Different spacing or line breaks.
- We restructure the messages so that the assistant's `content` field becomes a dictionary with separate `"sop"` and `"explanation"` keys.

✅ The reformatted dataset is saved to: `/content/drive/MyDrive/manasa_chatml_dataset_reformatted.jsonl`

In [ ]:
import json
import re

# Define paths
input_path = "/content/drive/MyDrive/manasa_chatml_dataset.jsonl"
output_path = "/content/drive/MyDrive/manasa_chatml_dataset_reformatted.jsonl"

def extract_sop_and_explanation(text):
    # Split on 🧠 Explanation or plain 'Explanation:' pattern
    parts = re.split(r"(?:\n\n|\\n\\n)?[\U0001f9e0🧠]?\s*Explanation[:：]?\s*", text, maxsplit=1)
    sop = parts[0].strip()
    explanation = parts[1].strip() if len(parts) > 1 else ""
    return sop, explanation

with open(input_path, "r") as infile, open(output_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)
        user_msg = data["messages"][0]
        assistant_msg = data["messages"][1]

        sop_text, explanation_text = extract_sop_and_explanation(assistant_msg["content"])

        new_data = {
            "messages": [
                user_msg,
                {
                    "role": "assistant",
                    "content": {
                        "sop": sop_text,
                        "explanation": explanation_text
                    }
                }
            ]
        }

        outfile.write(json.dumps(new_data) + "\n")

print("✅ Reformatted dataset saved to:", output_path)


✅ Reformatted dataset saved to: /content/drive/MyDrive/manasa_chatml_dataset_reformatted.jsonl
